In [1]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import Petna as pt
matplotlib.rcParams['figure.figsize'] = (50,50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 2000)
%reload_ext autoreload
%autoreload 2

In [2]:
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
sheets = xls.sheet_names
sheets

['Introduction',
 'Outcomes & Factors Rankings',
 'Outcomes & Factors SubRankings',
 'Ranked Measure Data',
 'Ranked Measure Sources & Years',
 'Additional Measure Data',
 'Addtl Measure Sources & Years']

In [3]:
# pt.missingvalues(sheets[5],xls)

In [4]:
rmd = xls.parse(sheets[3],header=1)
amd = xls.parse(sheets[5],header=1)
amd = amd.drop(columns=['State','County'])

## Creating our master dataFrame

In [5]:
df = rmd.merge(amd, on='FIPS')
# Focusing on percentage and rates as opposed to absolute values
for kw in ['95%','Quartile','#','Unreliable']:
    df = df.drop(columns=[x for x in df.columns if kw in x])
df = df.set_index('FIPS')

In [6]:
# Columns title formatting
subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),
        ('+','plus'), ('½','half'), ('-','_'), ('<','under'), ('%','percent'), ('/', '_or_') 
       ]
def col_formatting(col):
    for old, new in subs:
        col = col.replace(old,new)
    return col

df.columns = [col_formatting(col) for col in df.columns]

## Replacing NaN values by state average

In [7]:
df['MV_Mortality_Rate'] = df['MV_Mortality_Rate'].fillna(df.groupby(by='State')['MV_Mortality_Rate'].transform('mean'))

In [8]:
# Any NaN in MV_Mortality_Rate?
df[df['MV_Mortality_Rate'].isna()]

,State,County,Years_of_Potential_Life_Lost_Rate,YPLL_Rate_(Black),YPLL_Rate_(Hispanic),YPLL_Rate_(White),percent_Fair_or_Poor,Physically_Unhealthy_Days,Mentally_Unhealthy_Days,percent_LBW,percent_LBW_(Black),percent_LBW_(Hispanic),percent_LBW_(White),percent_Smokers,percent_Obese,Food_Environment_Index,percent_Physically_Inactive,percent_With_Access,percent_Excessive_Drinking,percent_Alcohol_Impaired,Chlamydia_Rate,Teen_Birth_Rate,Teen_Birth_Rate_(Black),Teen_Birth_Rate_(Hispanic),Teen_Birth_Rate_(White),percent_Uninsured_x,PCP_Rate,PCP_Ratio,Dentist_Rate,Dentist_Ratio,MHP_Rate,MHP_Ratio,Preventable_Hosp_Rate,Preventable_Hosp_Rate_(Black),Preventable_Hosp_Rate_(Hispanic),Preventable_Hosp_Rate_(White),percent_Screened,percent_Screened_(Black),percent_Screened_(Hispanic),percent_Screened_(White),percent_Vaccinated,percent_Vaccinated_(Black),percent_Vaccinated_(Hispanic),percent_Vaccinated_(White),Cohort_Size,Graduation_Rate,Population_x,percent_Some_College,Labor_Force,percent_Unemployed,percent_Children_in_Poverty,percent_Children_in_Poverty_(Black),percent_Children_in_Poverty_(Hispanic),percent_Children_in_Poverty_(White),80th_Percentile_Income,20th_Percentile_Income,Income_Ratio,percent_Single_Parent_Households,Association_Rate,Annual_Average_Violent_Crimes,Violent_Crime_Rate,Injury_Death_Rate,Average_Daily_PM25,Presence_of_violation,percent_Severe_Housing_Problems,Severe_Housing_Cost_Burden,Overcrowding,Inadequate_Facilities,percent_Drive_Alone,percent_Drive_Alone_(Black),percent_Drive_Alone_(Hispanic),percent_Drive_Alone_(White),percent_Long_Commute___Drives_Alone,Life_Expectancy,Life_Expectancy_(Black),Life_Expectancy_(Hispanic),Life_Expectancy_(White),Age_Adjusted_Mortality,Age_Adjusted_Mortality_(Black),Age_Adjusted_Mortality_(Hispanic),Age_Adjusted_Mortality_(White),Child_Mortality_Rate,Child_Mortality_Rate_(Black),Child_Mortality_Rate_(Hispanic),Child_Mortality_Rate_(White),Infant_Mortality_Rate,Infant_Mortality_Rate_(Black),Infant_Mortality_Rate_(Hispanic),Infant_Mortality_Rate_(White),percent_Frequent_Physical_Distress,percent_Frequent_Mental_Distress,percent_Diabetic,HIV_Prevalence_Rate,percent_Food_Insecure,percent_Limited_Access,Drug_Overdose_Mortality_Rate,MV_Mortality_Rate,percent_Insufficient_Sleep,percent_Uninsured_y,percent_Uninsured1,Other_PCP_Rate,Other_PCP_Ratio,percent_Disconnected_Youth,Household_Income,Household_income_(Black),Household_income_(Hispanic),Household_income_(White),percent_Free_or_Reduced_Lunch,Segregation_index,Segregation_Index,Homicide_Rate,Firearm_Fatalities_Rate,percent_Homeowners,percent_Severe_Housing_Cost_Burden,Population_y,percent_under_18,percent_65_and_over,percent_African_American,percent_American_Indian_or_Alaskan_Native,percent_Asian,percent_Native_Hawaiian_or_Other_Pacific_Islander,percent_Hispanic,percent_Non_Hispanic_White,percent_Not_Proficient_in_English,percent_Female,percent_Rural
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [9]:
df.to_csv('df.csv')